In [1]:
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np
import sqlite3

In [2]:
def divide(string):
    if 'https' in string:
        link = re.findall('<a href="([0-9a-z\./:-]*)">', string)[0]
        name = re.findall('>([A-ZÁÉÍÓÚÑa-záéíóúñüÜ ]*)<', string)[0].strip()
        return [name, link]
    else:
        name = re.findall('([A-ZÁÉÍÓÚÑa-záéíóúñ ]*)', string)[0].strip()
        return [name]

In [3]:
url = 'https://masteres.ugr.es/fisymat/docencia/plan-estudios'
page = urlopen(url)
html_bytes = page.read()
html = html_bytes.decode("utf-8")

# Subject names scraping

In [4]:
subjects_pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ:\n ]*)</td>'''
subjects = [x.replace('\n', '').strip() for x in re.findall(subjects_pattern, html)]
subjects

['Astrofísica y Cosmología',
 'Comunicación de la Astrofísica',
 'Física de Galaxias',
 'Física Estelar',
 'Radioastronomía',
 'Técnicas Observacionales en Astrofísica',
 'Análisis Numérico de Edp y Aproximación',
 'Bioestadística y Bioinformática',
 'Física de Redes Complejas y Aplicaciones Interdisciplinares',
 'Modelos Matemáticos en Ecología',
 'Movilidad y Dinámica Celular: Introducción a la Dinámica y Crecimiento Tumoral',
 'Seminario de Invitados Biomat y de Problemas Industriales en Biotecnología',
 'Análisis No Lineal y Ecuaciones Diferenciales',
 'Ecuaciones en Derivadas Parciales Dispersivas No Lineales',
 'Edp de Transporte en Teoría Cinética y Mecánica de Fluidos',
 'Métodos Avanzados de Análisis Funcional y Análisis de Fourier',
 'Principios de Geometría y Aplicaciones en Física',
 'Problemas Variacionales Geométricos',
 'Sistemas Dinámicos y Mecánica',
 'Topología Algebraica y Aplicaciones',
 'Desarrollos Actuales en Física Teórica y Matemáticas y su Fenomenología',
 'Fu

# Professors and e-mails scraped

In [5]:
elements = re.findall('<li>([a-zA-ZÁÉÍÓÚáéíóúñüÜ0-9 <>/:/\.\-\="\n]*)</li>', html)
elements = [x.strip().replace('\n', '') for x in elements]
elements = [re.split('</li> *<li>', x) for x in elements]
for i in range(len(elements)):
    elements.extend(elements[0])
    elements.remove(elements[0])
elements = [x.strip().replace('  ', ' ') for x in elements]
elements = elements[3:]
elements = [divide(x) for x in elements]
emails = []
for i in range(len(elements)):
    try:
        emails.append(elements[i][1])
    except:
        emails.append(np.nan)
        
names = pd.DataFrame([x[0] for x in elements], columns=['Name'])
emails = pd.DataFrame(emails, columns=['E-Mail'])

data = pd.concat([names, emails], axis=1)
data

,Name,E-Mail
0,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
...,...,...
59,Francisco Javier Lobillo Borrero,https://www.ugr.es/personal/francisco-javier-l...
60,Manuel Calixto Molina,https://www.ugr.es/personal/manuel-calixto-molina
61,Javier Brey Abalo,NaN
62,Juan Segundo Soler Vizcaino,https://www.ugr.es/personal/juan-segundo-soler...


# Complete transaction

In [6]:
pattern = '''<td>([A-Za-zÁÉÍÓÚáéíóúÑñ\n \.,:]*)</td>[0-9a-zA-ZáéíóúÁÉÍÓÚÑñüÜ<>/"=_\- ]*<ul> *<li>([a-zA-ZÁÉÍÓÚáéíóúñüÜ0-9 <>/:/\.\-\="\n]*)</li> *</ul>'''
result = re.findall(pattern, html.replace('\n', ''))
elements = []
for i in range(len(result)):
    elements.append(list(result[i]))
del result
for i in range(len(elements)):
    for j in range(len(elements[i])):
        elements[i][j] = elements[i][j].strip()
for i in range(len(elements)):
    elements[i][1] = re.split('</li> *<li>', elements[i][1])
for i in range(len(elements)):
    appender = []
    for j in range(len(elements[i][1])):
        appender.append(divide(elements[i][1][j]))
    elements[i][1] = appender
elements

[['Astrofísica y Cosmología',
  [['Mónica Relaño Pastor',
    'https://www.ugr.es/personal/monica-relano-pastor'],
   ['Ute Lisenfeld', 'https://www.ugr.es/personal/ute-lisenfeld']]],
 ['Comunicación de la Astrofísica',
  [['Jorge Jiménez Vicente',
    'https://www.ugr.es/personal/jorge-jimenez-vicente'],
   ['Tomás Ruiz Lara', 'https://www.ugr.es/personal/tomas-ruiz-lara']]],
 ['Física de Galaxias',
  [['Almudena Zurita Muñoz',
    'https://www.ugr.es/personal/almudena-zurita-munoz'],
   ['María Isabel Pérez Martín',
    'https://www.ugr.es/personal/maria-isabel-perez-martin']]],
 ['Física Estelar',
  [['Antonio García Hernández',
    'https://www.ugr.es/personal/antonio-garcia-hernandez'],
   ['Juan Carlos Suárez Yanes',
    'https://www.ugr.es/personal/juan-carlos-suarez-yanes']]],
 ['Radioastronomía',
  [['Angela Gardini'],
   ['Daniel Espada Fernández',
    'https://www.ugr.es/personal/daniel-espada-fernandez']]],
 ['Técnicas Observacionales en Astrofísica',
  [['Laura Sánchez Men

In [7]:
data = pd.DataFrame(columns=['Subject', 'Professor Name', 'Professor E-Mail'])
k = 0
for i in range(len(elements)):
    subject = elements[i][0]
    for j in range(len(elements[i][1])):
        try:
            email = elements[i][1][j][1]
        except:
            email = np.nan
        data.loc[k] = [subject, elements[i][1][j][0], email]
        k += 1
data

,Subject,Professor Name,Professor E-Mail
0,Astrofísica y Cosmología,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,Astrofísica y Cosmología,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,Comunicación de la Astrofísica,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,Comunicación de la Astrofísica,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,Física de Galaxias,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
...,...,...,...
59,Simetrías y Grupos de Lie en Física Matemática,Francisco Javier Lobillo Borrero,https://www.ugr.es/personal/francisco-javier-l...
60,Simetrías y Grupos de Lie en Física Matemática,Manuel Calixto Molina,https://www.ugr.es/personal/manuel-calixto-molina
61,Teoría Cinética,Javier Brey Abalo,NaN
62,Teoría Cinética,Juan Segundo Soler Vizcaino,https://www.ugr.es/personal/juan-segundo-soler...


## Save to database and read

In [9]:
with sqlite3.connect('data.db') as conn:
    data.to_sql('subjects_data', con=conn)

In [11]:
with sqlite3.connect('data.db') as conn:
    read_data = pd.read_sql('SELECT * FROM subjects_data', con=conn)
read_data

,index,Subject,Professor Name,Professor E-Mail
0,0,Astrofísica y Cosmología,Mónica Relaño Pastor,https://www.ugr.es/personal/monica-relano-pastor
1,1,Astrofísica y Cosmología,Ute Lisenfeld,https://www.ugr.es/personal/ute-lisenfeld
2,2,Comunicación de la Astrofísica,Jorge Jiménez Vicente,https://www.ugr.es/personal/jorge-jimenez-vicente
3,3,Comunicación de la Astrofísica,Tomás Ruiz Lara,https://www.ugr.es/personal/tomas-ruiz-lara
4,4,Física de Galaxias,Almudena Zurita Muñoz,https://www.ugr.es/personal/almudena-zurita-munoz
...,...,...,...,...
59,59,Simetrías y Grupos de Lie en Física Matemática,Francisco Javier Lobillo Borrero,https://www.ugr.es/personal/francisco-javier-l...
60,60,Simetrías y Grupos de Lie en Física Matemática,Manuel Calixto Molina,https://www.ugr.es/personal/manuel-calixto-molina
61,61,Teoría Cinética,Javier Brey Abalo,None
62,62,Teoría Cinética,Juan Segundo Soler Vizcaino,https://www.ugr.es/personal/juan-segundo-soler...
